In [1]:
#import xlrd
#book = xlrd.open_workbook('your.xlsx')
#sheet = book.sheet_by_name('example')
#data = [[sheet.cell_value(r, c) for c in range(sheet.ncols)] for r in range(sheet.nrows)]
#print data

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import xlrd
import os
import numpy as np
import time

In [2]:
DATA_FILE = '/home/hemanta/data/heart/heart_train_10.xls'
book = xlrd.open_workbook(DATA_FILE, encoding_override='utf-8')
sheet = book.sheet_by_index(0)
input_data = [[sheet.cell_value(r, c) for c in range(sheet.ncols-1)] for r in range(1, sheet.nrows)]
output_data = [[sheet.cell_value(r, 9) for r in range(1, sheet.nrows)]]

In [3]:
output_data[:1]

[[1.0,
  1.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,

In [4]:
X = tf.placeholder(tf.float32, [sheet.nrows-1, 9], name='X_placeholder')
Y = tf.placeholder(tf.float32, [1,sheet.nrows-1], name='Y_placeholder')

In [5]:
learning_rate = 0.001
n_epoch = 50

In [6]:
w = tf.Variable(tf.random_normal(shape=[9, 1], stddev=0.01), name='weights')
b = tf.Variable(tf.zeros([sheet.nrows-1, 1], name='bias'))

In [7]:
logits = tf.matmul(X, w) + b

In [8]:
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=tf.reshape(logits,[1, sheet.nrows-1]), labels=tf.reshape(output_data, [1, sheet.nrows-1]), name='loss')

In [9]:
#entropy2 = tf.sigmoid(entropy)

In [10]:
loss = tf.reduce_mean(entropy)

In [11]:
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

In [12]:
sess = tf.Session()
writer = tf.summary.FileWriter('./mygraph/03/heart', sess.graph)
start_time = time.time()
sess.run(tf.global_variables_initializer())
for i in range(n_epoch):
    total_loss = 0
    for _ in range(sheet.nrows-1):
        _, loss_batch = sess.run([optimizer, loss], feed_dict={X: input_data, Y: output_data})
        total_loss += loss_batch
    #print('Average loss epoch {0}: {1}'.format(i, total_loss/sheet.nrows))
    #print('Total_loss = {0}, loss_batch = {1}'.format(total_loss, loss_batch))
print('Total_time: {0} seconds'.format(time.time() - start_time))
print('Optimization Finished')
    

Total_time: 10.72080135345459 seconds
Optimization Finished


In [14]:
DATA_FILE = '/home/hemanta/data/heart/heart_test.xls'
book = xlrd.open_workbook(DATA_FILE, encoding_override='utf-8')
sheet2 = book.sheet_by_index(0)
input_data_test = [[sheet.cell_value(r, c) for c in range(sheet.ncols-1)] for r in range(1, sheet.nrows)]
output_data_test = [[sheet.cell_value(r, 9) for r in range(1, sheet.nrows)]]

In [15]:
sheet2.nrows

275

In [16]:
total_correct_preds = 0
#for i in range(sheet.nrows):
    #X_batch, Y_batch = mnist.test.next_batch(batch_size)
    #_ = sess.run([optimizer], feed_dict={X: X_batch, Y: Y_batch})
    #loss_batch = sess.run([loss], feed_dict={X: X_batch, Y: Y_batch})
    #logits_batch = sess.run([logits], feed_dict={X: X_batch, Y: Y_batch})
_, logits_batch = sess.run([optimizer, logits], feed_dict={X: input_data_test, Y: output_data_test})
preds = tf.nn.softmax(logits_batch)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(output_data_test, 1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
total_correct_preds += sess.run(accuracy)
    #print('correct pred {0}'.format(sess.run(correct_preds)))
print('Accuracy {0}'.format(total_correct_preds/sheet.nrows))
print('total correct preds {0} ,,, sheet no {1}'.format(total_correct_preds,sheet.nrows))
writer.close()

Accuracy 0.9963636363636363
total correct preds 274.0 ,,, sheet no 275
